# 同源策略

同源策略（Same origin policy）是一种约定, 它是浏览器最核心也最基本的安全功能,如果缺少了同源策略,则浏览器的正常功能可能都会受到影响.可以说Web是构建在同源策略基础之上的,浏览器只是针对同源策略的一种实现。

同源策略，它是由Netscape提出的一个著名的安全策略。现在所有支持JavaScript 的浏览器都会使用这个策略。所谓同源是指<font color="red">域名，协议，端口相同。</font>当一个浏览器的两个tab页中分别打开来 百度和谷歌的页面当浏览器的百度tab页执行一个脚本的时候会检查这个脚本是属于哪个页面的，即检查是否同源，只有和百度同源的脚本才会被执行。如果非同源，那么在请求数据时，浏览器会在控制台中报一个异常，提示拒绝访问。其实对应的服务端确实收到了请求，只不过是浏览器做了限制拿到数据也不展示而已(同源策略禁止读取位于xxx的远程资源)。所以我们经常看到在浏览器端发只会会带上对应域的cookie。

想想当我们访问了一个恶意网站 如果没有同源策略 那么这个网站就能通过js 访问document.cookie 得到用户关于的各个网站的sessionID 其中可能有银行网站。

# 解决跨域问题

目前解决跨域问题大致有2种方式：
1. jsonp   
2. cors

## ajax

### 没有指定contentType：默认是urlencode的方式发的


In [ ]:
<script>
    $(".send_ajax").click(function () {
         $.ajax({
            url:"/sendAjax/",
            type:"post",
            headers:{"X-CSRFToken":$.cookie('csrftoken')},   
            data:{
                num1:$(".num1").val(),
                num2:$(".num2").val(),
            },
            success:function (data) {
               alert(data);
               $(".ret").val(data)
            }
        })
    })

</script>

### 指定conentType为json数据发送

In [ ]:
<script>
    $(".send_ajax").click(function () {
         $.ajax({
            url:"/sendAjax/",
            type:"post",
            headers:{"X-CSRFToken":$.cookie('csrftoken')},   //如果是json发送的时候要用headers方式解决forbidden的问题
            data:JSON.stringify({
                num1:$(".num1").val(),
                num2:$(".num2").val()
            }),
            contentType:"application/json",  //客户端告诉浏览器是以json的格式发的数据，所以的吧发送的数据转成json字符串的形式发送
            success:function (data) {
               alert(data);
               $(".ret").val(data)
            }
        })
    })

</script>

In [ ]:
# views.py
def sendAjax(request):
    import json
    print(request.POST)  #<QueryDict: {}>
    print(request.GET)   #<QueryDict: {}>
    print(request.body)  #b'{"num1":"2","num2":"2"}'  注意这时的数据不再POST和GET里，而在body中
    print(type(request.body.decode("utf8")))  # <class 'str'>
    # 所以取值的时候得去body中取值，首先得反序列化一下
    data = request.body.decode("utf8")
    data = json.loads(data)
    num1 = data.get("num1")
    num2 = data.get("num2")
    ret = float(num1)+float(num2)
    return HttpResponse(ret)

基于jQuery的ajax和form发送的请求，都会默认有Content-Type，默认urlencode，
* Content-Type: 客户端告诉服务端我这次发送的数据是什么形式的
* dataType: 客户端期望服务端给我返回我设定的格式(jQuery内部会做相应的转换)

如果"Content-Type"="application/json"，发送的数据是对象形式的{},需要在body里面取数据，然后反序列化(按道理说django应该帮忙反序列化的，但是并没有，这也没有办法); 如果"Content-Type"="application/x-www-form-urlencoded"，发送的是/index/?name=haiyan&agee=20这样的数据，如果是POST请求需要在POST里取数据，如果是GET，在GET里面取数据.

## jsonp

将JSON数据填充进回调函数，这就是JSONP的JSON+Padding的含义。jsonp是json用来跨域的一个东西。原理是<font color="red">通过script标签的跨域特性来绕过同源策略。</font>


In [ ]:
<body>
<h1>项目一</h1>
<button class="send_jsonp">jsonp</button>
<script>
    $(".send_jsonp").click(function () {
        $.ajax({
            url:"",
            success:function (data) {
                console.log(data)
            }
        })
    });

    function func(arg) {
        console.log(arg)
    }
</script>
<script src="http://127.0.0.1:8080/ajax_send2/"></script>
</body>

In [ ]:
# views.py
def ajax_send2(request):
    import json
    print(222222)
    # return HttpResponse("func('name')")
    s = {"name":"haiyan","age":12}
    # return HttpResponse("func('name')")
    return HttpResponse("func('%s')"%json.dumps(s))   #返回一个func()字符串，正好自己的ajax里面有个func函数，就去执行func函数了，arg就是传的形参

JSONP的原型：创建一个回调函数，然后在远程服务上将JSON 数据形式作为参数传递给这个函数，将整个函数字符串返回，完成回调。

<font color="red">将JSON数据填充进回调函数，这就是JSONP的JSON+Padding的含义。</font>

在客户端定义的回调函数的函数名称传送给服务端，服务端则会返回以定义的回调函数名的方法，将获取的json数据传入这个方法完成回调



In [ ]:
<script>
    function addScriptTag(src){
         var script = document.createElement('script');
         script.setAttribute("type","text/javascript");
         script.src = src;
         document.body.appendChild(script);
         document.body.removeChild(script);
    }


    function func(name){
        alert("hello"+name)
    }

    function f(){
         addScriptTag("http://127.0.0.1:7766/SendAjax/callbacks=func")
    }
</script>

In [ ]:
# views.py
def SendAjax(request):
    import json
    dic={"k1":"v1"}
    print("callbacks:",request.GET.get("callbacks"))   
    callbacks=request.GET.get("callbacks")   #注意要在服务端得到回调函数名的名字
    return HttpResponse("%s('%s')"%(callbacks,json.dumps(dic)))

### jQuery对JSONP的实现

#### getJSON



In [ ]:
<button onclick="f()">sendAjax</button>

<script>

    function f(){
          $.getJSON("http://127.0.0.1:7766/SendAjax/?callbacks=?",function(arg){
            alert("hello"+arg)
        });  匿名函数
    }
    
</script>

要注意的是在url的后面必须添加一个callback参数，这样getJSON方法才会知道是用JSONP方式去访问服务，callback后面的那个？是内部自动生成的一个回调函数名。

此外，如果说我们想指定自己的回调函数名，或者说服务上规定了固定回调函数名该怎么办呢？我们可以使用$.ajax方法来实现.

#### $.ajax

In [ ]:
<script>
    function f(){
          $.ajax({
                url:"http://127.0.0.1:7766/SendAjax/",
                dataType:"jsonp",    
                jsonp: 'callbacks',   #键
　　　　　　　　　 jsonpCallback:"SayHi" #函数的名字
         });
    } 
    
    function SayHi(arg) { 
        alert(arg); 
    } 
</script>

**当然，最简单的形式还是通过回调函数来处理**


In [ ]:

<script>

    function f(){

            $.ajax({
               url:"http://127.0.0.1:7766/SendAjax/",
               dataType:"jsonp",            
               jsonp: 'callbacks',          //jQuery帮助随机生成的：callbacks="xxx"
               success:function(data){
                   alert("hi "+data)
              }
         });

       }

</script>

<font color="red">注意: JSONP一定是GET请求</font>

## CORS

CORS是一个W3C标准，全称是"跨域资源共享"（Cross-origin resource sharing）。它允许浏览器向跨源服务器，发出XMLHttpRequest请求，从而克服了AJAX只能同源使用的限制。

**CORS需要浏览器和服务器同时支持。整个CORS通信过程，都是浏览器自动完成，不需要用户参与。对于开发者来说，CORS通信与同源的AJAX通信没有差别，代码完全一样。浏览器一旦发现AJAX请求跨源，就会自动添加一些附加的头信息，有时还会多出一次附加的请求，但用户不会有感觉。因此，实现CORS通信的关键是服务器。只要服务器实现了CORS接口，就可以跨源通信。**

### 两种请求

浏览器将请求分成两类：简单请求（simple request）和非简单请求（not-so-simple request）。

只要同时满足以下两大条件，就属于简单请求。

1. 请求方法是以下三种方法之一：
    * HEAD
    * GET
    * POST
2. HTTP的头信息不超出以下几种字段：
    * Accept
    * Accept-Language
    * Content-Language
    * Last-Event-ID
    * Content-Type：只限于三个值application/x-www-form-urlencoded、multipart/form-data、text/plain.(其实就是form表单enctype参数的三个取值，但是通过ajax可以发送application/json格式的数据)。
    
凡是不同时满足上面两个条件，就属于非简单请求。

浏览器对这两种请求的处理，是不一样的。

#### 非简单请求

##### 预检请求

非简单请求是那种对服务器有特殊要求的请求，比如请求方法是PUT或DELETE，或者Content-Type字段的类型是application/json。

非简单请求的CORS请求，会在正式通信之前，增加一次HTTP查询请求，称为"预检"请求（preflight）。

浏览器先询问服务器，当前网页所在的域名是否在服务器的许可名单之中，以及可以使用哪些HTTP动词和头信息字段。只有得到肯定答复，浏览器才会发出正式的XMLHttpRequest请求，否则就报错。

```
Access-Control-Request-Headers: k1
Access-Control-Request-Method: PUT
Origin: http://127.0.0.1:9000
Referer: http://127.0.0.1:9000/books/12/23/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
```
"预检"请求用的请求方法是OPTIONS，表示这个请求是用来询问的。头信息里面，关键字段是Origin，表示请求来自哪个源。

除了Origin字段，"预检"请求的头信息包括两个特殊字段。

1. Access-Control-Request-Method: 该字段是必须的，用来列出浏览器的CORS请求会用到哪些HTTP方法，上例是PUT。
2. Access-Control-Request-Headers: 该字段是一个逗号分隔的字符串，指定浏览器CORS请求会额外发送的头信息字段，上例是k1。

##### 预检请求的回应

服务器收到"预检"请求以后，检查了Origin、Access-Control-Request-Method和Access-Control-Request-Headers字段以后，确认允许跨源请求，就可以做出回应。

```
Access-Control-Allow-Credentials: true
Access-Control-Allow-Headers: k1,k2
Access-Control-Allow-Methods: PUT,DELETE
Access-Control-Allow-Origin: http://127.0.0.1:9000
Content-Length: 7
Content-Type: text/html; charset=utf-8
Date: Tue, 16 Jul 2019 06:22:18 GMT
Server: WSGIServer/0.2 CPython/3.6.3
X-Frame-Options: SAMEORIGIN
```

如果浏览器否定了"预检"请求，会返回一个正常的HTTP回应，但是没有任何CORS相关的头信息字段。这时，浏览器就会认定，服务器不同意预检请求，因此触发一个错误，被XMLHttpRequest对象的onerror回调函数捕获。控制台会打印出如下的报错信息。

**CORS相关字段:**
1. Access-Control-Allow-Methods
    该字段必需，它的值是逗号分隔的一个字符串，表明服务器支持的所有跨域请求的方法。注意，返回的是所有支持的方法，而不单是浏览器请求的那个方法。这是为了避免多次"预检"请求。

2. Access-Control-Allow-Headers
    如果浏览器请求包括Access-Control-Request-Headers字段，则Access-Control-Allow-Headers字段是必需的。它也是一个逗号分隔的字符串，表明服务器支持的所有头信息字段，不限于浏览器在"预检"中请求的字段。

3. Access-Control-Allow-Credentials
    该字段与简单请求时的含义相同。

4. Access-Control-Max-Age
    该字段可选，用来指定本次预检请求的有效期，单位为秒。上面结果中，有效期是20天（1728000秒），即允许缓存该条回应1728000秒（即20天），在此期间，不用发出另一条预检请求。

5. Access-Control-Allow-Origin
    该字段是必须的。它的值要么是请求时Origin字段的值，要么是一个*，表示接受任意域名的请求。

6. Access-Control-Allow-Credentials
    该字段可选。它的值是一个布尔值，表示是否允许发送Cookie。默认情况下，Cookie不包括在CORS请求之中。设为true，即表示服务器明确许可，Cookie可以包含在请求中，一起发给服务器。这个值也只能设为true，如果服务器不要浏览器发送Cookie，删除该字段即可。

7. Access-Control-Expose-Headers
    该字段可选。CORS请求时，XMLHttpRequest对象的getResponseHeader()方法只能拿到6个基本字段：Cache-Control、Content-Language、Content-Type、Expires、Last-Modified、Pragma。如果想拿到其他字段，就必须在Access-Control-Expose-Headers里面指定。上面的例子指定，getResponseHeader('FooBar')可以返回FooBar字段的值。
    
### 跨域传输cookie

在跨域请求中，CORS请求默认不发送Cookie和HTTP认证信息。如果要把Cookie发到服务器，一方面要服务器同意，指定Access-Control-Allow-Credentials字段。另一方面，开发者必须在AJAX请求中打开withCredentials属性。

如果想要发送必须同时满足下面三个条件：
* 浏览器端：XMLHttpRequest的withCredentials为true
* 服务器端：Access-Control-Allow-Credentials为true
* 服务器端响应的 Access-Control-Allow-Origin 不能是通配符 *

如果要发送Cookie，Access-Control-Allow-Origin就不能设为星号，必须指定明确的、与请求网页一致的域名。同时，Cookie依然遵循同源政策，只有用服务器域名设置的Cookie才会上传，其他域名的Cookie并不会上传，且（跨源）原网页代码中的document.cookie也无法读取服务器域名下的Cookie。


In [ ]:
function JqSendRequest(){
            $.ajax({
                url: "http://127.0.0.1:8080/books/user",
                type: 'PUT',
                dataType: 'text',
                headers: {'k1': 'v1'},
                xhrFields:{withCredentials: true},
                success: function(data, statusText, xmlHttpRequest){
                    console.log(data);
                }
            })
        }

In [ ]:
# views.py
class UserIndex(View):
    def put(self, request,*args):
        res = HttpResponse("put")
        # res['Access-Control-Allow-Origin'] = "http://127.0.0.1:9000"

        # res['xxoo'] = "seven"
        # res['bili'] = "daobidao"
        # res['Access-Control-Expose-Headers'] = "xxoo,bili"

        # res.set_cookie('kkkkk', 'vvvvv')
        print(request.COOKIES)
        return res

    def options(self, *args, **kwargs):
        res = HttpResponse("options")
        res['Access-Control-Allow-Origin'] = "http://127.0.0.1:9000"
        res['Access-Control-Allow-Headers'] = "k1,k2"
        res['Access-Control-Allow-Methods'] = "PUT,DELETE"
        res['Access-Control-Allow-Credentials'] = "true"
        # res['Access-Control-Max-Age'] = 10
        return res